In [78]:
import numpy as np
import pandas as pd
from sklearn.linear_model import RidgeCV
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

possessions = pd.read_csv('shift_aggregated_20212022.csv')
possessions = possessions[possessions['timeofpos'] > 0]

In [108]:
teams = ['Boston Celtics', 'Philadelphia 76ers', 'Toronto Raptors', 'Brooklyn Nets']

pos_clean = possessions.loc[possessions['team1id'].isin(teams)].reset_index(drop=True)
pos_clean = pos_clean.loc[pos_clean['team2id'].isin(teams)].reset_index(drop=True)

In [109]:
pos_clean

,team1id,team1player1,team1player2,team1player3,team1player4,team1player5,team2id,team2player1,team2player2,team2player3,team2player4,team2player5,team1_points,team2_points,team1_turnovers,team2_turnovers,team1_rebounds,team2_rebounds,team1_blocks,team2_blocks,team1_steals,team2_steals,team1_fga,team2_fga,team1_fgm,team2_fgm,team1_3ptrsatt,team2_3ptrsatt,team1_3ptrsmade,team2_3ptrsmade,team1_fta,team2_fta,team1_ftmade,team2_ftmade,timeofpos
0,Boston Celtics,Al Horford,Marcus Smart,Jaylen Brown,Jayson Tatum,Grant Williams,Toronto Raptors,Pascal Siakam,Fred VanVleet,Svi Mykhailiuk,Precious Achiuwa,Scottie Barnes,18,17,3,4,6,6,0,0,0,0,12,14,6,6,4,9,2,4,0,0,0,0,156
1,Boston Celtics,Al Horford,Marcus Smart,Jaylen Brown,Jayson Tatum,Grant Williams,Toronto Raptors,Pascal Siakam,Fred VanVleet,Svi Mykhailiuk,Yuta Watanabe,Precious Achiuwa,11,16,3,0,4,6,0,0,0,0,9,12,4,6,5,5,3,2,0,0,0,0,65
2,Boston Celtics,Enes Freedom,Josh Richardson,Jayson Tatum,Romeo Langford,Payton Pritchard,Toronto Raptors,Pascal Siakam,Svi Mykhailiuk,Precious Achiuwa,Malachi Flynn,Scottie Barnes,8,0,2,0,6,1,0,0,0,0,2,6,1,0,2,2,1,0,0,0,0,0,66
3,Boston Celtics,Enes Freedom,Marcus Smart,Jayson Tatum,Romeo Langford,Payton Pritchard,Toronto Raptors,Pascal Siakam,Svi Mykhailiuk,Precious Achiuwa,Malachi Flynn,Justin Champagnie,2,0,0,0,1,1,0,1,0,0,3,1,1,0,0,0,0,0,0,0,0,0,28
4,Boston Celtics,Jaylen Brown,Jayson Tatum,Grant Williams,Al Horford,Marcus Smart,Toronto Raptors,Pascal Siakam,Fred VanVleet,Svi Mykhailiuk,Isaac Bonga,Scottie Barnes,7,5,0,0,2,2,0,0,0,0,3,3,1,1,3,3,1,1,0,0,0,0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,Toronto Raptors,Pascal Siakam,Svi Mykhailiuk,Gary Trent Jr.,Precious Achiuwa,Dalano Banton,Boston Celtics,Al Horford,Dennis Schroder,Josh Richardson,Romeo Langford,Grant Williams,5,3,1,3,3,1,2,0,0,2,4,3,1,1,0,2,0,1,0,0,0,0,38
259,Toronto Raptors,Pascal Siakam,Svi Mykhailiuk,Gary Trent Jr.,Precious Achiuwa,Dalano Banton,Boston Celtics,Josh Richardson,Jayson Tatum,Robert Williams III,Grant Williams,Marcus Smart,2,3,0,0,2,1,0,0,0,0,2,4,1,1,0,3,0,1,0,0,0,0,30
260,Toronto Raptors,Svi Mykhailiuk,Isaac Bonga,Malachi Flynn,Justin Champagnie,Dalano Banton,Boston Celtics,Juancho Hernangomez,Romeo Langford,Aaron Nesmith,Payton Pritchard,Enes Freedom,10,6,1,2,4,4,0,0,1,2,6,7,4,2,2,3,2,1,0,0,0,0,28
261,Toronto Raptors,Svi Mykhailiuk,Yuta Watanabe,Malachi Flynn,Justin Champagnie,Scottie Barnes,Brooklyn Nets,Nic Claxton,Kessler Edwards,Cam Thomas,Kevin Durant,Patty Mills,2,7,1,1,1,3,0,2,1,0,4,6,1,3,3,1,0,1,0,0,0,0,34


In [110]:
def build_player_list(posessions):
    players = list(
        set(list(posessions['team1player1'].unique()) + list(posessions['team1player2'].unique()) + list(
            posessions['team1player3']) + \
            list(posessions['team1player4'].unique()) + list(posessions['team1player5'].unique()) + list(
            posessions['team2player1'].unique()) + \
            list(posessions['team2player2'].unique()) + list(posessions['team2player3'].unique()) + list(
            posessions['team2player4'].unique()) + \
            list(posessions['team2player5'].unique())))
    players.sort()
    return players

In [111]:
player_list = build_player_list(pos_clean)

In [112]:
pos_clean['PointDifPerSec'] = (pos_clean['team1_points'] - pos_clean['team2_points']) / pos_clean['timeofpos']

In [83]:
# possessions['PointsPerSec'] = (possessions['team1_points'])  / possessions['timeofpos']


In [113]:
def map_players(row_in, players):
    p1 = row_in[0]
    p2 = row_in[1]
    p3 = row_in[2]
    p4 = row_in[3]
    p5 = row_in[4]
    p6 = row_in[5]
    p7 = row_in[6]
    p8 = row_in[7]
    p9 = row_in[8]
    p10 = row_in[9]

    rowOut = np.zeros([len(players) * 2])

    rowOut[players.index(p1)] = 1
    rowOut[players.index(p2)] = 1
    rowOut[players.index(p3)] = 1
    rowOut[players.index(p4)] = 1
    rowOut[players.index(p5)] = 1

    rowOut[players.index(p6) + len(players)] = -1
    rowOut[players.index(p7) + len(players)] = -1
    rowOut[players.index(p8) + len(players)] = -1
    rowOut[players.index(p9) + len(players)] = -1
    rowOut[players.index(p10) + len(players)] = -1

    return rowOut

In [114]:
# Break the dataframe into x_train (nxm matrix), y_train (nx1 matrix of target values), and weights (not necessary because all rows will have 1 possession)
def convert_to_matricies(possessions, name, players):
    # extract only the columns we need

    # Convert the columns of player ids into a numpy matrix
    stints_x_base = possessions[['team1player1', 'team1player2', 'team1player3', 'team1player4', 'team1player5', 
                        'team2player1', 'team2player2', 'team2player3', 'team2player4', 'team2player4']].to_numpy()
    
    # Apply our mapping function to the numpy matrix
    stint_X_rows = np.apply_along_axis(map_players, 1, stints_x_base, players)

    # Convert the column of target values into a numpy matrix
    stint_Y_rows = possessions[name].to_numpy()

    # extract the possessions as a pandas Series
    possessions = possessions['timeofpos']

    # return matricies and possessions series
    return stint_X_rows, stint_Y_rows, possessions

train_x, train_y, possessions_raw = convert_to_matricies(pos_clean, 'PointDifPerSec', player_list)

In [115]:
# Convert lambda value to alpha needed for ridge CV
def lambda_to_alpha(lambda_value, samples):
    return (lambda_value * samples) / 2.0

# Convert RidgeCV alpha back into a lambda value
def alpha_to_lambda(alpha_value, samples):
    return (alpha_value * 2.0) / samples

In [116]:
def calculate_rapm(train_x, train_y, possessions, lambdas, name, players):
    # convert our lambdas to alphas
    alphas = [lambda_to_alpha(l, train_x.shape[0]) for l in lambdas]

    # create a 5 fold CV ridgeCV model. Our target data is not centered at 0, so we want to fit to an intercept.
    clf = RidgeCV(alphas=alphas, cv=5, fit_intercept=True, normalize=False)

    # fit our training data
    model = clf.fit(train_x, train_y, sample_weight=possessions)

    # convert our list of players into a mx1 matrix
    player_arr = np.transpose(np.array(players).reshape(1, len(players)))
    # extract our coefficients into the offensive and defensive parts
    coef_offensive_array = np.transpose(np.array(model.coef_[ 0:len(players)]).reshape(1, len(players)))
    coef_defensive_array = np.transpose(np.array(model.coef_[len(players):]).reshape(1, len(players)))

    # concatenate the offensive and defensive values with the playey ids into a mx3 matrix
    player_id_with_coef = np.concatenate([player_arr, np.round(coef_offensive_array,4), np.round(coef_defensive_array,4)], axis = 1)
    # build a dataframe from our matrix
    players_coef = pd.DataFrame(player_id_with_coef)

    intercept = model.intercept_
    # apply new column names
    players_coef.columns = ['playerId', '{0}__Off'.format(name), '{0}__Def'.format(name)]

    # Add the offesnive and defensive components together (we should really be weighing this to the number of offensive and defensive possession played as they are often not equal).
    players_coef[name] = players_coef['{0}__Off'.format(name)].apply(lambda x: float(x)) + players_coef['{0}__Def'.format(name)].apply(lambda x: float(x))

    # rank the values
    players_coef['{0}_Rank'.format(name)] = players_coef[name].rank(ascending=False)
    players_coef['{0}__Off_Rank'.format(name)] = players_coef['{0}__Off'.format(name)].rank(ascending=False)
    players_coef['{0}__Def_Rank'.format(name)] = players_coef['{0}__Def'.format(name)].rank(ascending=False)

    # add the intercept for reference
    players_coef['{0}__intercept'.format(name)] = intercept

    return players_coef, intercept

In [117]:
lambdas_rapm = [.01, .05, .1]
results, intercept = calculate_rapm(train_x, train_y, possessions_raw, lambdas_rapm, 'PointDifPerSec', player_list)

# results = player_names.merge(results, how='inner', on='playerId')

In [118]:
results = np.round(results, decimals=2)
results

,playerId,RAPM__Off,RAPM__Def,RAPM,RAPM_Rank,RAPM__Off_Rank,RAPM__Def_Rank,RAPM__intercept
0,Aaron Henry,-0.0555,0.0,-0.06,51.0,46.0,31.5,-0.03
1,Aaron Nesmith,0.0,0.0373,0.04,20.0,33.0,11.0,-0.03
2,Al Horford,0.0629,-0.0396,0.02,28.0,8.0,50.0,-0.03
3,Andre Drummond,0.035,0.0546,0.09,6.0,16.0,6.0,-0.03
4,Blake Griffin,-0.001,-0.0469,-0.05,50.0,61.0,45.0,-0.03
...,...,...,...,...,...,...,...,...
57,Shake Milton,0.0124,-0.0119,0.00,35.0,27.0,56.0,-0.03
58,Svi Mykhailiuk,-0.0381,-0.0001,-0.04,46.0,50.0,62.0,-0.03
59,Tobias Harris,-0.0474,-0.0717,-0.12,59.0,47.0,42.0,-0.03
60,Tyrese Maxey,-0.0026,0.0201,0.02,32.0,60.0,16.0,-0.03


In [119]:
results.sort_values('RAPM_Rank').head(25)

,playerId,RAPM__Off,RAPM__Def,RAPM,RAPM_Rank,RAPM__Off_Rank,RAPM__Def_Rank,RAPM__intercept
38,Khem Birch,0.1282,0.0,0.13,1.0,1.0,31.5,-0.03
50,Paul Reed,0.0,0.121,0.12,2.0,33.0,1.0,-0.03
9,Chris Boucher,0.0288,0.0884,0.12,3.0,20.0,2.0,-0.03
27,James Johnson,0.0953,0.0,0.10,4.0,2.0,31.5,-0.03
14,DeAndre' Bembry,0.0932,0.0,0.09,5.0,3.0,31.5,-0.03
3,Andre Drummond,0.035,0.0546,0.09,6.0,16.0,6.0,-0.03
8,Charles Bassey,0.0877,0.0,0.09,7.0,5.0,31.5,-0.03
53,Robert Williams III,0.0,0.0704,0.07,8.0,33.0,3.0,-0.03
12,David Duke Jr.,0.0,0.0674,0.07,9.0,33.0,4.0,-0.03
19,Gary Trent Jr.,0.0388,0.0257,0.06,10.0,14.0,14.0,-0.03


In [91]:
results.to_csv('data/rapm.csv')
print(results)

          playerId RAPM__Off RAPM__Def  RAPM  RAPM_Rank  RAPM__Off_Rank  RAPM__Def_Rank  RAPM__intercept
0      Aaron Henry   -0.0555       0.0 -0.06       51.0            46.0            31.5            -0.03
1    Aaron Nesmith       0.0    0.0373  0.04       20.0            33.0            11.0            -0.03
2       Al Horford    0.0629   -0.0396  0.02       28.0             8.0            50.0            -0.03
3   Andre Drummond     0.035    0.0546  0.09        6.0            16.0             6.0            -0.03
4    Blake Griffin    -0.001   -0.0469 -0.05       50.0            61.0            45.0            -0.03
..             ...       ...       ...   ...        ...             ...             ...              ...
57    Shake Milton    0.0124   -0.0119  0.00       35.0            27.0            56.0            -0.03
58  Svi Mykhailiuk   -0.0381   -0.0001 -0.04       46.0            50.0            62.0            -0.03
59   Tobias Harris   -0.0474   -0.0717 -0.12       59.0